# Import Libraries

In [1]:
# drive access
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# standard libraries
import numpy as np
import pandas as pd

In [22]:
# for extracting countries and continents
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
!pip install pycountry_convert
import pycountry_convert as pc

# Load train and test csv files

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/project/train_df.csv')

train_df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,duration
0,comsan,[],"['flight call', 'nocturnal flight call']",50.7542,4.5672,Actitis hypoleucos,Common Sandpiper,Frederik Fluyt,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/587730,comsan/XC587730.ogg,5.746937
1,eaywag1,[],"['call', 'sex uncertain']",69.3585,88.2378,Motacilla flava,Western Yellow Wagtail,Alexander Hellquist,Creative Commons Attribution-NonCommercial-Sha...,3.0,http://xeno-canto.org/675944,eaywag1/XC675944.ogg,5.355000
2,comsan,[],"['life stage uncertain', 'nocturnal flight cal...",41.1698,0.9761,Actitis hypoleucos,Common Sandpiper,Xavier Riera,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/664012,comsan/XC664012.ogg,10.488000
3,barswa,[],"['alarm call', 'flight call']",NaN,NaN,Hirundo rustica,Barn Swallow,Daniel Parker,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/268804,barswa/XC268804.ogg,76.538813
4,barswa,[],['song'],53.9299,-2.9833,Hirundo rustica,Barn Swallow,Chris Batty,Creative Commons Attribution-NonCommercial-Sha...,2.5,http://xeno-canto.org/690498,barswa/XC690498.ogg,83.095500


In [4]:
test_df = pd.read_csv('/content/drive/MyDrive/project/test_df.csv')

test_df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename,duration
0,barswa,[],[''],53.2509,5.5980,Hirundo rustica,Barn Swallow,Gosse Hoekstra,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/721711,barswa/XC721711.ogg,19.069375
1,comsan,[],"['flight call', 'nocturnal flight call']",48.8306,2.1999,Actitis hypoleucos,Common Sandpiper,Stanislas Wroza,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/496602,comsan/XC496602.ogg,28.995938
2,eaywag1,[],[''],43.3298,4.8364,Motacilla flava,Western Yellow Wagtail,Stanislas Wroza,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/718445,eaywag1/XC718445.ogg,7.340438
3,eaywag1,"['cohmar1', 'hoopoe']","['call', 'flight call']",37.1357,-7.6138,Motacilla flava,Western Yellow Wagtail,Nelson Conceição,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/481360,eaywag1/XC481360.ogg,151.944000
4,barswa,[],[''],19.3551,-99.0467,Hirundo rustica,Barn Swallow,Manuel Grosselet,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/698512,barswa/XC698512.ogg,18.703688


# 1. Inspect each column for any NAN values

In [5]:
for column in train_df.columns:
  print(column)
  print(train_df[column].isnull().sum())
  print('-'*50)

primary_label
0
--------------------------------------------------
secondary_labels
0
--------------------------------------------------
type
0
--------------------------------------------------
latitude
17
--------------------------------------------------
longitude
17
--------------------------------------------------
scientific_name
0
--------------------------------------------------
common_name
0
--------------------------------------------------
author
0
--------------------------------------------------
license
0
--------------------------------------------------
rating
0
--------------------------------------------------
url
0
--------------------------------------------------
filename
0
--------------------------------------------------
duration
0
--------------------------------------------------


# 2. Inspect each column for any outliers or anything needs attention

1. clean-up type column
2. use latitude and longitude to find the country and continents
3. turn rating column to binary

In [6]:
for column in train_df.columns:
  print(train_df[column].value_counts())
  print('-'*100)

comsan     352
eaywag1    351
barswa     341
Name: primary_label, dtype: int64
----------------------------------------------------------------------------------------------------
[]                        1021
['wlwwar']                   7
['cohmar1']                  4
['barswa']                   3
['eubeat1']                  2
['woosan']                   2
['spwlap1']                  1
['bkctch1']                  1
['thrnig1']                  1
['eaywag1']                  1
['chibat1', 'reftin1']       1
Name: secondary_labels, dtype: int64
----------------------------------------------------------------------------------------------------
['flight call']                                                        149
['call']                                                               124
['']                                                                   120
['song']                                                               105
['nocturnal flight call']                

# 3. Drop unused columns

In [7]:
drop_columns = ['secondary_labels', 'scientific_name', 'common_name', 'author', 'license', 'url']

In [8]:
train_df.drop(drop_columns, axis=1, inplace=True)

train_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration
0,comsan,"['flight call', 'nocturnal flight call']",50.7542,4.5672,4.0,comsan/XC587730.ogg,5.746937
1,eaywag1,"['call', 'sex uncertain']",69.3585,88.2378,3.0,eaywag1/XC675944.ogg,5.355000
2,comsan,"['life stage uncertain', 'nocturnal flight cal...",41.1698,0.9761,5.0,comsan/XC664012.ogg,10.488000
3,barswa,"['alarm call', 'flight call']",NaN,NaN,2.5,barswa/XC268804.ogg,76.538813
4,barswa,['song'],53.9299,-2.9833,2.5,barswa/XC690498.ogg,83.095500


In [9]:
test_df.drop(drop_columns, axis=1, inplace=True)

test_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration
0,barswa,[''],53.2509,5.5980,4.0,barswa/XC721711.ogg,19.069375
1,comsan,"['flight call', 'nocturnal flight call']",48.8306,2.1999,4.0,comsan/XC496602.ogg,28.995938
2,eaywag1,[''],43.3298,4.8364,4.0,eaywag1/XC718445.ogg,7.340438
3,eaywag1,"['call', 'flight call']",37.1357,-7.6138,4.5,eaywag1/XC481360.ogg,151.944000
4,barswa,[''],19.3551,-99.0467,5.0,barswa/XC698512.ogg,18.703688


# 4. Clean up the type column

### for train_df

In [10]:
train_df['type'].value_counts().head(10)

['flight call']                                                       149
['call']                                                              124
['']                                                                  120
['song']                                                              105
['nocturnal flight call']                                              95
['call', 'flight call']                                                55
['flight call', 'nocturnal flight call']                               24
['male', 'song']                                                       24
['life stage uncertain', 'nocturnal flight call', 'sex uncertain']     20
['call', 'song']                                                       18
Name: type, dtype: int64

In [11]:
train_df.loc[train_df['type'].str.contains('call') & train_df['type'].str.contains('song'), 'type'] = 'both'
train_df.loc[train_df['type'].str.contains('call'), 'type'] = 'call'
train_df.loc[train_df['type'].str.contains('song'), 'type'] = 'song'
train_df.loc[train_df['type'] == "['']", 'type'] = 'blank'

In [12]:
train_df['type'].value_counts().head(10)

call                                         689
song                                         154
blank                                        120
both                                          74
['uncertain']                                  2
['Adults feeding babies']                      1
['Single Note Calls']                          1
['life stage uncertain', 'sex uncertain']      1
['Song']                                       1
['adult', 'sex uncertain']                     1
Name: type, dtype: int64

In [13]:
train_df.loc[train_df['type'].str.contains('Song'), 'type'] = 'song'

In [14]:
train_df['type'].value_counts().head(10)

call                                         689
song                                         155
blank                                        120
both                                          74
['uncertain']                                  2
['Adults feeding babies']                      1
['Single Note Calls']                          1
['life stage uncertain', 'sex uncertain']      1
['adult', 'sex uncertain']                     1
Name: type, dtype: int64

In [15]:
train_df.loc[(train_df['type'] != 'call') & (train_df['type'] != 'song') & (train_df['type'] != 'blank') & (train_df['type'] != 'both'), 'type'] = 'blank'

In [16]:
train_df['type'].value_counts().head(10)

call     689
song     155
blank    126
both      74
Name: type, dtype: int64

### for test_df

In [17]:
test_df['type'].value_counts().head(10)

['flight call']                                                       67
['call']                                                              64
['']                                                                  51
['song']                                                              40
['nocturnal flight call']                                             39
['call', 'flight call']                                               15
['male', 'song']                                                      14
['call', 'female', 'flight call', 'male', 'song']                     13
['alarm call']                                                        11
['life stage uncertain', 'nocturnal flight call', 'sex uncertain']     9
Name: type, dtype: int64

In [18]:
test_df.loc[test_df['type'].str.contains('call') & test_df['type'].str.contains('song'), 'type'] = 'both'
test_df.loc[test_df['type'].str.contains('call'), 'type'] = 'call'
test_df.loc[test_df['type'].str.contains('song'), 'type'] = 'song'
test_df.loc[test_df['type'] == "['']", 'type'] = 'blank'

In [19]:
test_df['type'].value_counts().head(10)

call             282
song              66
blank             51
both              45
['uncertain']      2
['alarm']          1
['juvenile']       1
Name: type, dtype: int64

In [20]:
test_df.loc[(test_df['type'] != 'call') & (test_df['type'] != 'song') & (test_df['type'] != 'blank') & (test_df['type'] != 'both'), 'type'] = 'blank'

In [21]:
test_df['type'].value_counts().head(10)

call     282
song      66
blank     55
both      45
Name: type, dtype: int64

# 5. Extract countries and continents using latitude and longitude

In [23]:
latitude = train_df['latitude'].iloc[0]
longitude = train_df['longitude'].iloc[0]
print(latitude)
print(longitude)

50.7542
4.5672


In [24]:
geolocator = Nominatim(user_agent="project", timeout=10)
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
location = geocode([latitude, longitude], language='en')
location.raw

{'place_id': 121646963,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 814662289,
 'lat': '50.75418485',
 'lon': '4.567242807048872',
 'class': 'building',
 'type': 'house',
 'place_rank': 30,
 'importance': 9.99999999995449e-06,
 'addresstype': 'building',
 'name': '',
 'display_name': '12, Kerkstraat, Tombeek, Overijse, Halle-Vilvoorde, Flemish Brabant, Flanders, 3090, Belgium',
 'address': {'house_number': '12',
  'road': 'Kerkstraat',
  'village': 'Tombeek',
  'town': 'Overijse',
  'county': 'Halle-Vilvoorde',
  'state': 'Flemish Brabant',
  'ISO3166-2-lvl6': 'BE-VBR',
  'region': 'Flanders',
  'ISO3166-2-lvl4': 'BE-VLG',
  'postcode': '3090',
  'country': 'Belgium',
  'country_code': 'be'},
 'boundingbox': ['50.7541484', '50.7542444', '4.5671715', '4.5673157']}

In [25]:
location.raw['address']['country_code']

'be'

### create reusable functions

In [26]:
def extract_countries(locations_list):
  countries = []

  for location in locations_list:
    geolocator = Nominatim(user_agent="project", timeout=10)
    geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    if np.isnan(location[0]) or np.isnan(location[1]):
      country = 'Unknown'
    else:
      locate = geocode(location, language='en')
      if locate is None:
        country = 'Antarctica'
      elif 'address' in locate.raw and 'country_code' in locate.raw['address']:
        country = locate.raw['address']['country_code']
      else:
        country = 'Unknown'

    country = country.upper()
    countries.append(country)

  assert len(countries) == len(locations_list)
  return countries

In [35]:
def extract_continents(countries_list):
  continents = []

  for country in countries_list:
    if country == 'UNKNOWN':
      continent = 'UNKNOWN'
    else:
      continent = pc.country_alpha2_to_continent_code(country)

    continents.append(continent)

  assert len(continents) == len(countries_list)
  return continents

### for train_df

In [28]:
train_locations = train_df[['latitude','longitude']].values
train_locations

array([[50.7542,  4.5672],
       [69.3585, 88.2378],
       [41.1698,  0.9761],
       ...,
       [58.3026, 26.7361],
       [59.3794, 24.2407],
       [43.7412, -7.867 ]])

In [29]:
train_countries = extract_countries(train_locations)
train_countries

['BE',
 'RU',
 'ES',
 'UNKNOWN',
 'GB',
 'MX',
 'SE',
 'RU',
 'GB',
 'MX',
 'HN',
 'US',
 'GB',
 'GB',
 'BA',
 'US',
 'IT',
 'GB',
 'DE',
 'SE',
 'ES',
 'IE',
 'DE',
 'PL',
 'PL',
 'FR',
 'FR',
 'FR',
 'ES',
 'NL',
 'PL',
 'FI',
 'NO',
 'GB',
 'RU',
 'UNKNOWN',
 'RU',
 'ES',
 'GB',
 'ES',
 'FR',
 'ES',
 'SE',
 'ES',
 'FR',
 'GB',
 'UNKNOWN',
 'FI',
 'FR',
 'FR',
 'BE',
 'FI',
 'UNKNOWN',
 'GB',
 'ES',
 'ES',
 'CN',
 'ES',
 'FR',
 'NO',
 'ES',
 'ES',
 'ES',
 'IE',
 'DE',
 'ES',
 'PL',
 'ES',
 'IE',
 'GB',
 'RU',
 'PT',
 'SE',
 'NO',
 'PT',
 'RU',
 'ES',
 'EE',
 'IE',
 'CA',
 'OM',
 'UNKNOWN',
 'FI',
 'RU',
 'RU',
 'PL',
 'GB',
 'PL',
 'GB',
 'DE',
 'RU',
 'GE',
 'NL',
 'FR',
 'HR',
 'BE',
 'GB',
 'BE',
 'ES',
 'RU',
 'SE',
 'RU',
 'GB',
 'MX',
 'GB',
 'DE',
 'EE',
 'IT',
 'CY',
 'GB',
 'ES',
 'EE',
 'PT',
 'GB',
 'MX',
 'RU',
 'PL',
 'GB',
 'DE',
 'ES',
 'DK',
 'IE',
 'RU',
 'SE',
 'RU',
 'HR',
 'US',
 'RU',
 'GB',
 'US',
 'FR',
 'PL',
 'IL',
 'AT',
 'CN',
 'GB',
 'NL',
 'EE',
 'RU',
 '

In [30]:
assert len(train_countries) == len(train_df)

In [31]:
train_df['country'] = train_countries

In [32]:
train_df['country'].value_counts()

FR         138
ES         124
GB          96
RU          88
PL          60
PT          51
SE          48
DE          47
NL          40
IE          30
FI          26
US          25
BE          24
UA          19
KZ          17
UNKNOWN     17
AT          16
MX          15
EE          14
IT          13
CN          13
NO          12
DK          11
HR           9
UZ           9
ZA           7
CA           5
CH           5
GR           4
GE           4
BG           4
TR           4
MN           4
TH           3
BR           3
IN           3
CY           3
JP           3
ZM           2
TW           2
HU           2
CD           2
SG           2
SN           2
HN           2
ID           2
IR           1
KW           1
CV           1
OM           1
AR           1
IL           1
AE           1
MA           1
BO           1
BA           1
LA           1
PA           1
KR           1
LV           1
Name: country, dtype: int64

In [34]:
train_df[['primary_label', 'latitude', 'longitude', 'country']][train_df['country']=='UNKNOWN']

,primary_label,latitude,longitude,country
3,barswa,NaN,NaN,UNKNOWN
35,barswa,NaN,NaN,UNKNOWN
46,barswa,NaN,NaN,UNKNOWN
52,barswa,NaN,NaN,UNKNOWN
81,barswa,NaN,NaN,UNKNOWN
150,barswa,NaN,NaN,UNKNOWN
213,barswa,NaN,NaN,UNKNOWN
312,barswa,NaN,NaN,UNKNOWN
317,barswa,NaN,NaN,UNKNOWN
358,barswa,NaN,NaN,UNKNOWN


In [36]:
train_continents = extract_continents(train_countries)
train_continents

['EU',
 'EU',
 'EU',
 'UNKNOWN',
 'EU',
 'NA',
 'EU',
 'EU',
 'EU',
 'NA',
 'NA',
 'NA',
 'EU',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'UNKNOWN',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'UNKNOWN',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'UNKNOWN',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'NA',
 'AS',
 'UNKNOWN',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'AS',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 '

In [38]:
assert len(train_continents) == len(train_df)

In [39]:
train_df['continent'] = train_continents

In [40]:
train_df['continent'].value_counts()

EU         883
AS          76
NA          48
UNKNOWN     17
AF          15
SA           5
Name: continent, dtype: int64

In [42]:
train_df.loc[train_df['continent'] == 'NA', 'continent'] = 'AMERICAS'
train_df.loc[train_df['continent'] == 'SA', 'continent'] = 'AMERICAS'
train_df.loc[train_df['continent'] == 'EU', 'continent'] = 'EUROPE'
train_df.loc[train_df['continent'] == 'AS', 'continent'] = 'ASIA'
train_df.loc[train_df['continent'] == 'AF', 'continent'] = 'AFRICA'

In [43]:
train_df['continent'].value_counts()

EUROPE      883
ASIA         76
AMERICAS     53
UNKNOWN      17
AFRICA       15
Name: continent, dtype: int64

In [44]:
train_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent
0,comsan,call,50.7542,4.5672,4.0,comsan/XC587730.ogg,5.746937,BE,EUROPE
1,eaywag1,call,69.3585,88.2378,3.0,eaywag1/XC675944.ogg,5.355000,RU,EUROPE
2,comsan,call,41.1698,0.9761,5.0,comsan/XC664012.ogg,10.488000,ES,EUROPE
3,barswa,call,NaN,NaN,2.5,barswa/XC268804.ogg,76.538813,UNKNOWN,UNKNOWN
4,barswa,song,53.9299,-2.9833,2.5,barswa/XC690498.ogg,83.095500,GB,EUROPE


### for test_df

In [45]:
test_locations = test_df[['latitude','longitude']].values
test_locations

array([[  53.2509  ,    5.598   ],
       [  48.8306  ,    2.1999  ],
       [  43.3298  ,    4.8364  ],
       [  37.1357  ,   -7.6138  ],
       [  19.3551  ,  -99.0467  ],
       [  56.0851  ,   47.2602  ],
       [  48.7448  ,   -1.5638  ],
       [  43.8335  ,   87.5906  ],
       [  52.4965  ,   10.1638  ],
       [  13.8949  ,  100.4655  ],
       [  44.1947  ,   87.8477  ],
       [  44.1062  ,    1.8827  ],
       [  43.2891  ,   -8.6749  ],
       [  44.9351  ,   12.4225  ],
       [  49.0715  , -106.5309  ],
       [  61.5473  ,   29.5888  ],
       [  41.4875  ,    2.3653  ],
       [  44.6967  ,   34.3473  ],
       [  46.3315  ,   -1.3214  ],
       [  51.3532  ,   -0.1461  ],
       [  48.7221  ,   -3.5738  ],
       [  55.55    ,   10.63    ],
       [  49.4434  ,   21.9175  ],
       [  53.411   ,   -1.5665  ],
       [  37.0951  ,   -6.0386  ],
       [  42.3285  ,    2.5512  ],
       [  51.8061  ,   16.6513  ],
       [  39.6216  ,   -0.2614  ],
       [  54.2707  ,

In [46]:
test_countries = extract_countries(test_locations)
test_countries

['NL',
 'FR',
 'FR',
 'PT',
 'MX',
 'RU',
 'FR',
 'CN',
 'DE',
 'TH',
 'CN',
 'FR',
 'ES',
 'IT',
 'CA',
 'FI',
 'ES',
 'UA',
 'FR',
 'GB',
 'FR',
 'DK',
 'PL',
 'GB',
 'ES',
 'ES',
 'PL',
 'ES',
 'DE',
 'SE',
 'IN',
 'ES',
 'RU',
 'FR',
 'NL',
 'PL',
 'RU',
 'NL',
 'FR',
 'IE',
 'UA',
 'SE',
 'PT',
 'FR',
 'ES',
 'FR',
 'ES',
 'TH',
 'SE',
 'RU',
 'BE',
 'RU',
 'GB',
 'HR',
 'RU',
 'FR',
 'ES',
 'GB',
 'ES',
 'ES',
 'RU',
 'PL',
 'RU',
 'UNKNOWN',
 'RU',
 'FR',
 'MX',
 'FR',
 'UA',
 'MX',
 'FR',
 'ES',
 'DE',
 'FR',
 'ES',
 'ES',
 'SE',
 'FR',
 'UA',
 'IE',
 'ES',
 'IE',
 'SE',
 'RU',
 'GB',
 'RU',
 'UA',
 'PL',
 'RU',
 'IE',
 'UA',
 'FR',
 'NL',
 'SA',
 'FR',
 'ES',
 'GB',
 'PL',
 'FR',
 'ES',
 'GB',
 'CN',
 'US',
 'IN',
 'PL',
 'FR',
 'ES',
 'ES',
 'NL',
 'ES',
 'FR',
 'CN',
 'ES',
 'RU',
 'FR',
 'GB',
 'GB',
 'GB',
 'RU',
 'FI',
 'SE',
 'RU',
 'SE',
 'PL',
 'AR',
 'IL',
 'US',
 'RU',
 'PL',
 'DE',
 'ES',
 'GB',
 'KZ',
 'SE',
 'DE',
 'BE',
 'BE',
 'RU',
 'SE',
 'RU',
 'FR',
 'ZA',
 

In [47]:
assert len(test_countries) == len(test_df)

In [48]:
test_df['country'] = test_countries

In [49]:
test_df['country'].value_counts()

ES         66
FR         54
RU         48
GB         33
PL         24
DE         19
SE         18
PT         17
BE         17
NL         14
UA         13
IE         12
US         10
FI          9
MX          9
DK          7
UNKNOWN     6
CN          6
NO          5
EE          5
AT          5
KZ          5
IT          5
IN          3
CA          3
HR          3
SA          2
UZ          2
BR          2
TW          2
CH          2
HU          2
LK          2
ZA          2
TH          2
AR          2
MY          2
JP          1
RO          1
TR          1
GE          1
CZ          1
KR          1
MA          1
GR          1
IL          1
IR          1
Name: country, dtype: int64

In [50]:
test_df[['primary_label', 'latitude', 'longitude', 'country']][test_df['country']=='UNKNOWN']

,primary_label,latitude,longitude,country
63,comsan,NaN,NaN,UNKNOWN
163,barswa,NaN,NaN,UNKNOWN
178,barswa,NaN,NaN,UNKNOWN
193,barswa,NaN,NaN,UNKNOWN
288,barswa,NaN,NaN,UNKNOWN
400,barswa,NaN,NaN,UNKNOWN


In [51]:
test_continents = extract_continents(test_countries)
test_continents

['EU',
 'EU',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'AS',
 'EU',
 'AS',
 'AS',
 'EU',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'UNKNOWN',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'NA',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'NA',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'SA',
 'AS',
 'NA',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AS',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'EU',
 'AF',
 

In [52]:
assert len(test_continents) == len(test_df)

In [53]:
test_df['continent'] = test_continents

In [54]:
test_df['continent'].value_counts()

EU         381
AS          32
NA          22
UNKNOWN      6
SA           4
AF           3
Name: continent, dtype: int64

In [55]:
test_df.loc[test_df['continent'] == 'NA', 'continent'] = 'AMERICAS'
test_df.loc[test_df['continent'] == 'SA', 'continent'] = 'AMERICAS'
test_df.loc[test_df['continent'] == 'EU', 'continent'] = 'EUROPE'
test_df.loc[test_df['continent'] == 'AS', 'continent'] = 'ASIA'
test_df.loc[test_df['continent'] == 'AF', 'continent'] = 'AFRICA'

In [56]:
test_df['continent'].value_counts()

EUROPE      381
ASIA         32
AMERICAS     26
UNKNOWN       6
AFRICA        3
Name: continent, dtype: int64

In [57]:
test_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent
0,barswa,blank,53.2509,5.5980,4.0,barswa/XC721711.ogg,19.069375,NL,EUROPE
1,comsan,call,48.8306,2.1999,4.0,comsan/XC496602.ogg,28.995938,FR,EUROPE
2,eaywag1,blank,43.3298,4.8364,4.0,eaywag1/XC718445.ogg,7.340438,FR,EUROPE
3,eaywag1,call,37.1357,-7.6138,4.5,eaywag1/XC481360.ogg,151.944000,PT,EUROPE
4,barswa,blank,19.3551,-99.0467,5.0,barswa/XC698512.ogg,18.703688,MX,AMERICAS


# Save cleaned train and test dfs to csv

In [58]:
train_df.to_csv('/content/drive/MyDrive/project/clean_train_df.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/project/clean_test_df.csv', index=False)